In [2]:
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import os



#Dosya yolundan YOLO klasörü yükleniyor
print("[BILGI]  Veriler Alınıyor")
MODEL_PATH = r"C:\Users\MONSTER\Untitled Folder 3\yolo-coco"



MIN_CONF = 0.3    # minimum nesne algılama güvenini
NMS_THRESH = 0.3  # maksimala olmayan bastırma eşiği tanımlanır

#NVIDIA CUDA GPU'nun kullanılıp kullanılılmaması gerektiğini gösteren boole değeri, kullanmayacağımız için false
USE_GPU = False

#iki kişinin birbirinden olabileceği minimum güvenli mesafe (piksel cinsinden)
#insanların sosyal mesafeye uymak için birbirlerinden kalmaları gereken minimum mesafe (piksel cinsinden)
MIN_DISTANCE = 50

#insanları algılamak için fonksiyon oluşturuldu
print("[BILGI]  Kişileri Algılama Fonksiyonu")
def detect_people(frame, net, ln, personIdx=0):
    #frame/çerçeve: Video dosyanızdan veya doğrudan web kamera ağınızdan 
    #net/kare: Önceden başlanmış ve önceden eğitilmiş YOLO nesne algılama modeli 
    #ln: YOLO CNN çıkış katmanı adları
    #personIdx: YOLO modeli birçok nesne türünü algılayabilir; Bu dizin özellikle kişi sınıfı içindir, 
    #çünkü diğer nesneleri dikkate almayacağız
    
    
    (H, W) = frame.shape[:2] #ölçekleme amacıyla çerçeve boyutlarını yakalanır.
    results = [] #sonuç listesi başlatılır Sonuçlar;
    #(1) kişi tahmin olasılığı, 
    #(2) algılama için sınırlayıcı kutu koordinatları ve (3) nesnenin merkezkoid oluşur.
    
    #Çerçevemiz göz önüne alındığında YOLO ile çıkarım yapma işlemi
    #Derin sinir ağlarına sahip nesneleri doğru şekilde tahmin etmek için, verilerimizi önceden işlememiz gerekir ve 
    #cv2.dnn modülü bize bu amaçla iki işlev sağlar: blobFromImage ve blobFromImages. 
    #Bu işlevler, isteğe bağlı olarak ölçekleme, ortalama çıkarma ve kanal değiştirme gerçekleştirir. 
    
    #Çerçevemizi önceden işlemek için bir blob oluşturuyoruz.
    
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    #Buradan YOLO ve OpenCV ile nesne algılama gerçekleştirebiliyoruz.
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    #listeler oluşturuluyor
    boxes = [] # sınırlayıcı kutularımız
    centroids = [] #nesne centroidlerimiz
    confidences = [] #nesne algılama güvenlerimizi 

    #çıktıların her biri üzerinde döngü başlatılıyor
    for output in layerOutputs:
        #algılamaların her biri üzerinde döngü başlatılıyor
        for detection in output:
            #önce geçerli algılanan nesnenin classID'sini ve güvenini (yani olasılık) ayıklarız.
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
            #Buradan, (1) geçerli algılamanın bir kişi olduğunu ve (2) minimum güvenin karşılandığını veya aşıldığını doğrularız.
            if classID == personIdx and confidence > MIN_CONF:
                # Bunu varsayarsak, sınırlayıcı kutu koordinatları görüntünün boyutuna göre  hesaplanır ve sonra sınırlayıcı 
                #kutunun merkezini (yani centroid) türetiyoruz. YOLO sınırlayıcı kutunun orta (x, y) koordinatlarını 
                #ve ardından kutuların genişliğini ve yüksekliğini döndürr
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                
                
                #listeler güncelleniyor bounding box koordinatları,
                # centroids, and confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))
    
    #Zayıf, çakışan sınırlayıcı kutuları bastırmak için non-maxima bastırma uygulanıyor.
    #non-maxima bastırma: birçok örtüşen varlık arasından tek bir varlık seçen bir bilgisayarlı görme yöntemidir 
    #Maksimal olmayan bastırmanın amacı zayıf, çakışan sınırlayıcı kutuları bastırmaktır.
    #(OpenCV için yerleşiktir) ve algılamalar idx'leri ile sonuçlanır.
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)
    
    #non-maxima bastırmanın en az bir algılama sağladığı kontrol ediliyor
    if len(idxs) > 0:
        #tuttuğumuz dizinler üzerinde döngü başlatılıyor.
        for i in idxs.flatten():
            #boundbox sınırlayıcı kutu koordinatları elde ediliyor.
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            #her kişi tespitinin güvenliği, sınırlayıcı kutusu ve merkezi güncelleniyor. kişinin merkezi elde ediliyor.
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    # return the list of results
    return results


print("[BILGI] YOLO diskten yükleniyor..")
#YOLO modelinin eğittiği COCO sınıf etiketleri dosya yolundan yükleniyor
labelsPath = r"C:\Users\MONSTER\Untitled Folder 3\yolo-coco\coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

print("Yolo detection category : ",LABELS[0])

#YOLO yolları (YOLO weight ve YOLO configuration) dosya yolundan yükleniyor
weightsPath = r"C:\Users\MONSTER\Untitled Folder 3\yolo-coco\yolov3.weights"
configPath = r"C:\Users\MONSTER\Untitled Folder 3\yolo-coco\yolov3.cfg"

#YOLO yolları kullanılarak model belleğe yükleniyor

#COCO veri kümesinde eğitilmiş (80 sınıf) YOLO nesne dedektörü yükleniyor
#OpenCV'nin dnn modülü yardımıyla YoloV3 ağırlıklarını ve konfigürasyon dosyaları yükleniyor.
print("[INFO] YOLO diskten yükleniyor...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

#YOLO'dan yalnızca ihtiyacımız olan *output* katman adlarını belirleniyor
#YOLO'dan çıktı katmanlarının indislerini topluyoruz; Sonuçlarımızı işlemek için onlara ihtiyacımız olacak.
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


#video akışı ve işaretçi video dosyasının çıktısını almak için başlatılıyor
#video akışımızı başlatıyoruz.
print("[INFO] video akışına erişiyor...")
#işlenecek video
cap = cv2.VideoCapture(r"D:\OneDrive Dosyaları\OneDrive\Masaüstü\mac2_Trim.mp4")

writer = None

#Son olarak, çerçeveleri işlemeye ve insanların güvenli sosyal mesafeyi koruyup korumadığını belirlemeye hazırız:
#video akışından kareler üzerinde döngü kuruluyor.
while True:
    
    #dosyadan sonraki kareyi okuyarak, kareler üzerinde döngü başlar
    (grabbed, frame) = cap.read()
    
     #çerçeve alınmamışsa, akışın sonuna ulaştık
    if not grabbed:
        break
    
    # resize the frame and then detect people (and only people) in it
    frame = imutils.resize(frame, width=700) #tahmin için çerçeve yeniden boyutlandırılıyor.
    results = detect_people(frame, net, ln,
                            personIdx=LABELS.index("person"))
    # initialize the set of indexes that violate the minimum social
    # distance
    violate = set()
    
    #en az iki kişi algılama kontrolu
    if len(results) >= 2:
        
        #Tüm centroid çiftleri arasındaki Öklid mesafesini hesaplama
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")
        
        #mesafe matrisinin üst üçgeni üzerinde döngü
        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                #Mesafenin belirlenen minimum sosyal mesafemizi ihlal edip etmediği kontrol ediliyor. 
                #İki kişi çok yakınsa, bunları ihlal kümesine ekleriz
                if D[i, j] < MIN_DISTANCE:
                    #ihlal setimizi centroid çiftlerinin dizinleriyle güncelleyin
                    violate.add(i)
                    violate.add(j)
    
    #çözüm üzerinde döngü başlatılıyor
    for (i, (prob, bbox, centroid)) in enumerate(results):
        #Sınırlayıcı kutuyu ve centroid koordinatlarını ayıklanır sonra açıklamanın rengini başlatılır
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid
        color = (0, 255, 0)
        #Geçerli dizinin ihlal kümemizde bulunup bulunmadığına bakılır ve varsa rengi kırmızıya güncelleyin
        if i in violate:
            color = (0, 0, 255)
        # draw (1) a bounding box around the person and (2) the
        # centroid coordinates of the person,
        #Hem kişinin sınırlayıcı kutusunu hem de nesne centroidini çizilir. 
        #Her biri renk koordineli, bu yüzden hangi insanların çok yakın olduğunu göreceğiz.
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.circle(frame, (cX, cY), 5, color, 1)
    # Toplam sosyal mesafe ihlali sayısı hakkında bilgi görüntüleyin 
    
    text = "Sosyal Mesafe ihlali: {}".format(len(violate))
    cv2.putText(frame, text, (10, frame.shape[0] - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)
    
    #çıktı frame olarak gösterilmeye hazır
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
    if writer is None:
        #frame mp4 video olarak dosyaya kaydediliyor
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        writer = cv2.VideoWriter(r'C:\Users\MONSTER\Untitled Folder 3\outputs\futbol.mp4', fourcc, 25,
                                 (frame.shape[1], frame.shape[0]), True)
    
    if writer is not None:
        writer.write(frame)    

cap.release()
writer.release()
cv2.destroyAllWindows()





[BILGI]  Veriler Alınıyor
[BILGI]  Kişileri Algılama Fonksiyonu
[BILGI] YOLO diskten yükleniyor..
Yolo detection category :  person
[INFO] YOLO diskten yükleniyor...
[INFO] video akışına erişiyor...
